In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

In [3]:
num_train = 100 # number of training data
num_feat = 4 # size of feature
num_test = 10000
deg = 1.0 # polynomial degree
e = 1.0 # scale of normal std or the range of uniform. For the error term
num_arcs = 10

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = num_arcs # num of arcs
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_With_2_Nodes_0401_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_coef_seed="+str(coef_seed)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_With_2_Nodes_0401_DDR_Data_Generation/


## Function to generate data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,version = 0) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, iter,version = 0) 
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # #  ****** Data generation *********
        thres = num_test
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, thres, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 

        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

## Compute out-of-sample cost

In [6]:
def compute_oracle_cost(cost_test):
    data_size = np.shape(cost_test)[0]
    _cost = np.zeros(data_size)
    for j in range(data_size):
        _cost[j] = np.min(cost_test[j,:])
    return _cost

In [7]:
def compute_out_of_sample_cost(W,w0,cost_test,feature):
    data_size = np.shape(feature)[0]
    _cost = np.zeros(data_size)
    for j in range(data_size):
        cost = W @ feature[j,:] + w0
        opt_index = np.argmin(cost)
        cost_test_tem = cost_test[j,:]
        _cost[j] = cost_test_tem[opt_index]
    return _cost

# Oracle

In [8]:
iteration_all = np.arange(0,10)
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

In [9]:
W_star_all[3]

array([[0.5507979 , 0.70814782, 0.29090474, 0.51082761],
       [0.89294695, 0.89629309, 0.12558531, 0.20724288],
       [0.0514672 , 0.44080984, 0.02987621, 0.45683322],
       [0.64914405, 0.27848728, 0.6762549 , 0.59086282],
       [0.02398188, 0.55885409, 0.25925245, 0.4151012 ],
       [0.28352508, 0.69313792, 0.44045372, 0.15686774],
       [0.54464902, 0.78031476, 0.30636353, 0.22195788],
       [0.38797126, 0.93638365, 0.97599542, 0.67238368],
       [0.90283411, 0.84575087, 0.37799404, 0.09221701],
       [0.6534109 , 0.55784076, 0.36156476, 0.2250545 ]])

In [10]:
cost_Oracle_all = {}; cost_Oracle_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    # cost_Oracle_all[iter] = compute_oracle_cost(c_test_all[iter])
    cost_Oracle_all[iter] = compute_out_of_sample_cost(W_star_all[iter],np.ones(num_arcs)*bump,c_test_all[iter],x_test_all[iter])
    cost_Oracle_avg[iter] = np.nanmean(cost_Oracle_all[iter])

## OLS

In [11]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}; cost_OLS_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_OLS_all[iter] = compute_out_of_sample_cost(W_ols_all[iter],w0_ols_all[iter],c_test_all[iter],x_test_all[iter])
    cost_OLS_avg[iter] = np.nanmean(cost_OLS_all[iter])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


## DDR

In [12]:
def solve_DDR(mu,lamb,num_acrs,num_feat,num_train,c_input,x_input):
    model = ro.Model()            # create an RSOME model
    w0_d = model.dvar(num_acrs)
    W_d = model.dvar((num_acrs,num_feat))

    alpha = model.dvar(num_train)                   

    eps = model.dvar(num_train*num_acrs)                   
    eps_index = 0
    for i in range(num_train):
        for j in range(num_acrs):
            model.st(alpha[i] >= -mu * c_input[i,j] - (1 - mu) * (W_d[j,:] @ x_input[i,:] + w0_d[j]))
            model.st(eps[eps_index] == c_input[i,j] - W_d[j,:] @ x_input[i,:] - w0_d[j])
            eps_index = eps_index + 1
    model.min(rso.norm(eps) + sum(alpha) * (lamb/num_train)) 

    model.solve(cpt,display=False,log=False)
    # print("status=",model.solution.status)
    w0_d_sol = w0_d.get()
    W_d_sol = W_d.get()
    obj = model.obj()
    return w0_d_sol,W_d_sol,obj

In [13]:
def Loss(x_train, z_train, W, w0):
    #W and w0 can be a tuplelist or an array
#     x_train = data[3]
#     z_train = data[5]
    N,p = x_train.shape
    N,d = z_train.shape
    a = []
    for n in range(N):
        for i in range(d):
            temp = []
            for j in range(p):
                temp.append(x_train[n][j]*W[i,j])
            a.append((z_train[n][i] - sum(temp) - w0[i])*(z_train[n][i] - sum(temp) - w0[i]))      
    return np.sum(a)/N

In [14]:
def ddr_solver(x_train, z_train, thres, mu, lamb, yconstraint = 0, A = 0, b = 0, yB = 0, B = 0):
#     x_train = data[3]
    z_train_min = np.minimum(z_train,thres) 
    N,p = x_train.shape
    N,d = z_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)] )
    t_ind = tuplelist( [n for n in range(N)] )
    
    W_ddr = m.addVars( W_ind, lb=-GRB.INFINITY )
    w0_ddr = m.addVars( w0_ind, lb=-GRB.INFINITY )
    t_ddr = m.addVars( t_ind, lb=-GRB.INFINITY )
    
    if yconstraint == 0:
        m.setObjective( Loss(x_train, z_train, W_ddr, w0_ddr) + lamb*(quicksum([t_ddr[n]  for n in range(N)])/ N) , GRB.MINIMIZE)

        if yB == 0:
            m.addConstrs( (- mu*z_train[n,i] - (1- mu)*(quicksum(x_train[n][j]*W_ddr[i,j] for j in range(p)) + w0_ddr[i])-\
                        t_ddr[n] <= 0 for n in range(N) for i in range(d)) )

            # m.addConstrs( (-mu*z_train[n,i] -(1- mu)*thres - t_ddr[n] <= 0 for n in range(N) for i in range(d)) )
        else:
            d,h = B.shape
            m.addConstrs( (- mu*quicksum(B[i,k]*z_train_min[n,k] for k in range(h)) \
                        - (1- mu)*(quicksum(B[i,k]*(quicksum(x_train[n][j]*W_ddr[k,j] for j in range(p)) + w0_ddr[k]) \
                                            for k in range(h))) - t_ddr[n] <= 0 for n in range(N) for i in range(d)) )

            m.addConstrs( (-mu*quicksum(B[i,k]*z_train_min[n,k] for k in range(h)) \
                        -(1- mu)*thres*quicksum(B[i,k] for k in range(h)) - t_ddr[n] <= 0 for n in range(N) for i in range(d)) )
    else:
        a,d = A.shape
        beta_ind = tuplelist( [(n,k) for n in range(N) for k in range(a)] )
        beta_ddr = m.addVars( beta_ind, lb= 0 )
        
        m.setObjective( Loss(x_train, z_train, W_ddr, w0_ddr) +\
                    lamb*(quicksum([t_ddr[n] + quicksum(beta_ddr[n,k]*b[k] for k in range(a))  for n in range(N)])/ N), GRB.MINIMIZE)


        m.addConstrs( (- mu*z_train_min[n,i] - (1- mu)*(quicksum(x_train[n][j]*W_ddr[i,j] for j in range(p)) + w0_ddr[i])-\
                    t_ddr[n] - quicksum(A[k,i]*beta_ddr[n,k] for k in range(a)) <= 0 for n in range(N) for i in range(d)) )

        m.addConstrs( (-mu*z_train_min[n,i] -(1- mu)*thres - t_ddr[n] - quicksum(A[k,i]*beta_ddr[n,k] for k in range(a)) <= 0 for n in range(N) for i in range(d)) )

    m.optimize()

    W = m.getAttr('x', W_ddr)
    w0 = m.getAttr('x', w0_ddr)
    t = m.getAttr('x', t_ddr)
    W_results = []
    for i in range(d):
        W_results.append([W[(i,j)] for j in range(p)])
    w0_results = [w0[i] for i in range(d)]
#     t_results = [t[n] for n in range(N)]
    return W_results, w0_results

In [22]:
mu_all = np.arange(0.2,0.3,0.01)
lamb_all = np.round(np.arange(0.2,0.3,0.01),4)

W_ddr_all = {}; w0_ddr_all = {}; obj_ddr_all = {}
cost_DDR_all = {}; cost_DDR_avg_all = {}
for iter in iteration_all:
    c_input = c_train_all[iter]
    x_input = x_train_all[iter]
    
    cost_ddr_avg_tem = np.zeros((len(mu_all),len(lamb_all)))
    for mu_index in range(len(mu_all)):
        mu = mu_all[mu_index]
        for lamb_index in range(len(lamb_all)):
            lamb = lamb_all[lamb_index]
            # print("iter = ",iter,",mu = ",mu,",lamb=",lamb)
            # w0_ddr_all[iter,mu,lamb],W_ddr_all[iter,mu,lamb],obj_ddr_all[iter,mu,lamb] = solve_DDR(mu,lamb,d,num_feat,num_train,c_input,x_input)
            W_ddr_all[iter,mu,lamb], w0_ddr_all[iter,mu,lamb] = ddr_solver(x_input, c_input, num_test, mu, lamb)
            cost_DDR_all[iter,mu,lamb] = compute_out_of_sample_cost(W_ddr_all[iter,mu,lamb],w0_ddr_all[iter,mu,lamb],c_test_all[iter],x_test_all[iter])
            cost_ddr_avg_tem[mu_index,lamb_index] = np.nanmean(cost_DDR_all[iter,mu,lamb])
    print("iter = ",iter,",cost_ddr_avg_tem")
    print(np.round(cost_ddr_avg_tem/cost_OLS_avg[iter],4))
    cost_DDR_avg_all[iter] = cost_ddr_avg_tem

iter =  0 ,cost_ddr_avg_tem
[[0.9998 0.9997 0.9997 0.9997 0.9996 0.9997 0.9996 0.9997 0.9997 0.9997]
 [0.9998 0.9997 0.9997 0.9997 0.9996 0.9996 0.9996 0.9996 0.9996 0.9996]
 [1.     0.9997 0.9998 0.9998 0.9998 0.9998 0.9997 0.9996 0.9996 0.9996]
 [1.     0.9999 0.9998 0.9999 0.9998 0.9998 0.9998 0.9996 0.9996 0.9996]
 [1.     0.9999 0.9999 0.9999 0.9998 0.9998 0.9997 0.9996 0.9996 0.9996]
 [1.     1.     0.9999 0.9998 0.9998 0.9998 0.9998 0.9997 0.9996 0.9996]
 [1.     1.     0.9999 0.9999 0.9999 0.9998 0.9998 0.9998 0.9996 0.9996]
 [1.0001 1.0001 0.9999 0.9999 0.9999 0.9998 0.9999 0.9999 0.9998 0.9997]
 [1.0001 1.0001 1.     0.9999 0.9999 0.9999 0.9999 0.9999 0.9999 0.9997]
 [1.     1.0001 1.0001 0.9999 0.9999 0.9998 0.9999 0.9998 1.     0.9998]]
iter =  1 ,cost_ddr_avg_tem
[[0.9997 0.9996 0.9997 0.9996 0.9996 0.9996 0.9995 0.9996 0.9996 0.9996]
 [0.9997 0.9997 0.9997 0.9996 0.9996 0.9996 0.9995 0.9996 0.9996 0.9996]
 [0.9998 0.9998 0.9999 0.9998 0.9998 0.9998 0.9998 0.9999 0.9998 0.

In [23]:
regret_DDR_vs_OLS_avg = np.zeros((len(mu_all),len(lamb_all)))
for iter in iteration_all:
    regret_DDR_vs_OLS_avg = regret_DDR_vs_OLS_avg + (cost_OLS_avg[iter] - cost_DDR_avg_all[iter])/(cost_OLS_avg[iter] - cost_Oracle_avg[iter])
regret_DDR_vs_OLS_avg = regret_DDR_vs_OLS_avg/len(iteration_all)

In [24]:
print("regret_DDR_vs_OLS_avg=")
print(np.round(regret_DDR_vs_OLS_avg*100,4))

regret_DDR_vs_OLS_avg=
[[1.9214 2.6017 2.7393 2.5311 3.1318 3.4522 3.6545 3.6208 3.8888 3.9261]
 [1.7546 1.9402 2.293  2.2591 2.5726 3.1191 3.4287 3.392  3.8534 3.6412]
 [1.2092 1.2817 1.2817 1.3925 1.8033 2.3948 2.7436 2.8342 3.648  3.496 ]
 [1.363  1.0771 1.587  1.5122 1.6037 1.957  1.9844 2.7692 3.0173 3.0406]
 [0.9143 0.9943 1.3427 1.406  1.2567 1.4919 1.8498 2.2859 2.3749 2.2451]
 [1.09   1.1472 1.1172 1.3042 1.6164 1.567  1.7244 2.3214 2.308  2.8732]
 [1.2276 1.4435 1.3794 1.8188 1.4538 1.551  1.9442 2.339  2.9124 3.3034]
 [1.5904 1.5736 2.3322 2.1648 2.2542 1.9647 2.1069 2.0361 2.7468 3.2062]
 [1.7549 1.7147 2.2847 2.189  2.1464 2.1448 1.8874 1.7176 2.0176 2.856 ]
 [2.0154 1.8368 2.0784 2.2892 2.3283 2.1936 1.7065 1.6561 1.8699 2.4386]]
